<a href="https://colab.research.google.com/github/Kid7ho/LLM-Assignment/blob/main/Assignment_3_2022148033.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
### Jin-Myoung Hyun 2022148033

## **Setting Up**: Setting up the environment before experiment

- Server address : `https://cot.ngrok.app/`
- Endpoints:
    - `/api/inference`: For calling API to generate outputs from LMs
    - `/api/change_password`: Change your password. Arguments: `student_id`, `old_password`, `new_password`.

- Check your remaining credit at this page.[https://cot.ngrok.app/student_credits]

### Step 0-1. Changing the password

In [8]:
import requests
import os

server_address = "https://cot.ngrok.app"
data = {
    "student_id": "2022148033",
    "old_password": "helloworld!",
    "new_password": "Helloworld!"
}
response = requests.post(f'{server_address}/api/change_password', json=data)

# Check if the request was successful
if response.status_code == 200:
    # Set the new password as an environment variable
    os.environ["INTRO_AI_API_KEY"] = data["new_password"]
    print("Environment variable INTRO_AI_API_KEY set successfully.")
else:
    print("Failed to change password. Status code:", response.status_code)

Environment variable INTRO_AI_API_KEY set successfully.


In [20]:
class LLM:
    def __init__(self, model_name, student_id, password):
        self.student_id = student_id
        self.password = password
        server_address = "https://cot.ngrok.app"
        self.api_url = f'{server_address}/api/inference'  # Replace with your server address
        self.model_name = model_name

    def generate(self, model_input, max_tokens=50, top_p=1.0, temperature=0.7, frequency_penalty=0, stop="\n\n"):
        data = {
            'model_name': self.model_name,
            'model_input': model_input,
            'student_id': self.student_id,
            'password': self.password,
            'max_tokens': max_tokens,
            'top_p': top_p,
            'temperature': temperature,
            'frequency_penalty': frequency_penalty,
            'stop': stop
        }
        response = requests.post(self.api_url, json=data)

        if response.status_code == 200:
            return response.json()  # Assuming the server returns a JSON response
        else:
            return f"Error: {response.text}"

In [21]:
student_id = "2022148033"
password = os.environ['INTRO_AI_API_KEY']

available_models = [
    "meta-llama/Llama-2-7b-hf",
    "facebook/opt-350m",
    "facebook/opt-1.3b",
    "facebook/opt-2.7b"
]
opt = LLM(available_models[1], student_id, password)

### Step 0-2. Load GSM8K dataset

In [22]:
!pip install datasets

In [23]:
from datasets import load_dataset

gsm8k = load_dataset("gsm8k", "main")['test']

### Step 0-3. Check an example from GSM8k

In [24]:
print("Question:")
print(gsm8k['question'][0])
print("="*100)
print("Answer:")
print(gsm8k['answer'][0])
#Answer는 #### 뒤에 나옴으로 답만 구하려면 split해서 뒤에 element를 선택하면 됨

Question:
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Answer:
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


## **Experiment 1**: Zero-shot prompting v.s. Few-shot prompting

### Step 1-1. Constructing prompts

In [25]:
import random
random.seed(0)

def construct_prompt(num_exemplars):
    # Load train set of GSM8K
    gsm8k = load_dataset("gsm8k", "main")['train']

    sampled_indices = random.sample([i for i in range(len(gsm8k['question']))], num_exemplars)


    instruction = "Instruction:\nSolve the following mathematical question and generate the answer after a tag, 'Answer:'."

    # Constructing a prompt with few-shot demonstrations from GSM8K
    prompt = instruction
    for i in range(num_exemplars):
        cur_question = gsm8k['question'][i]
        cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += f"\n[Example {num_exemplars+1}]\n"
    prompt += "Question:\n{question}\nAnswer:"

    # Write the prompt to a .txt file
    with open(f"prompt_{num_exemplars}shot.txt", "w") as f:
        f.write(prompt)

construct_prompt(0)
construct_prompt(5)
construct_prompt(10)

### Step 1-2. Load the prompt for zero-shot prompting

In [35]:
with open("prompt_0shot.txt", "r") as f:
    prompt_0shot = f.read()

from tqdm import tqdm
import json

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []

#너무 많음으로 앞에 50개만 사용 (과제를 할 때도 50개만 사용하면 됨)
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_0shot.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n", "[Example 2]"])
    if "Error:" in result:
        print(result)
        break
    cur_prediction = result['generations'][0]['text']
    # print(cur_prediction)

    pass_collected.append(cur_prediction.strip().replace("$", "0") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_0shot_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:00<00:38,  1.27it/s]

Acc: 0.0


  4%|▍         | 2/50 [00:02<00:58,  1.22s/it]

Acc: 0.0


  6%|▌         | 3/50 [00:03<00:49,  1.06s/it]

...

Acc: 0.0


 98%|█████████▊| 49/50 [01:32<00:03,  3.82s/it]

Acc: 0.0


100%|██████████| 50/50 [01:33<00:00,  1.88s/it]

Acc: 0.0


### Step 1-3. Load the Prompt for five-shot prompting

In [42]:
with open("prompt_5shot.txt", "r") as f:
    prompt_5shot = f.read()

from tqdm import tqdm

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_5shot.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break
    cur_prediction = result['generations'][0]['text']
    # print(cur_prediction)

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_5shot_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:01<00:51,  1.05s/it]

Acc: 0.0


  4%|▍         | 2/50 [00:02<00:55,  1.16s/it]

Acc: 0.0


  6%|▌         | 3/50 [00:03<00:57,  1.23s/it]

...

Acc: 0.020833333333333332


 98%|█████████▊| 49/50 [01:03<00:01,  1.39s/it]

Acc: 0.02040816326530612


100%|██████████| 50/50 [01:05<00:00,  1.30s/it]

Acc: 0.02


### Step 1-4. Load the Prompt for ten-shot prompting

In [43]:
with open("prompt_10shot.txt", "r") as f:
    prompt_10shot = f.read()

from tqdm import tqdm

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_10shot.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break
    cur_prediction = result['generations'][0]['text']
    # print(cur_prediction)

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_10shot_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:01<01:20,  1.63s/it]

Acc: 0.0


  4%|▍         | 2/50 [00:03<01:20,  1.69s/it]

Acc: 0.0


  6%|▌         | 3/50 [00:04<01:16,  1.62s/it]

...

Acc: 0.020833333333333332


 98%|█████████▊| 49/50 [01:13<00:01,  1.55s/it]

Acc: 0.02040816326530612


100%|██████████| 50/50 [01:15<00:00,  1.50s/it]

Acc: 0.02


## **Experience 2**. Comparing Standard Prompting and CoT Prompting

### Step 2-1. Constructing Prompts

In [58]:
import random
random.seed(0)

def construct_prompt_CoT(num_exemplars):
    # Load train set of GSM8K
    gsm8k = load_dataset("gsm8k", "main")['train']

    sampled_indices = random.sample([i for i in range(len(gsm8k['question']))], num_exemplars)


    instruction = "Instruction:\nSolve the following mathematical question and generate the answer after a tag, 'Answer:'."

    # Constructing a prompt with few-shot demonstrations from GSM8K
    prompt = instruction
    for i in range(num_exemplars):
        cur_question = gsm8k['question'][i]
        cur_answer = gsm8k['answer'][i].replace("####", "The answer is").strip()
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += f"\n[Example {num_exemplars+1}]\n"
    prompt += "Question:\n{question}\nAnswer:"

    # Write the prompt to a .txt file
    with open(f"prompt_{num_exemplars}shot_CoT.txt", "w") as f:
        f.write(prompt)

construct_prompt_CoT(5)
construct_prompt_CoT(10)

### Step 2-2. Implementing CoT to five-shot Prompting

In [65]:
with open("prompt_5shot_CoT.txt", "r") as f:
    prompt_5shot_CoT = f.read()

from tqdm import tqdm

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_5shot_CoT.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break

    tmp = result['generations'][0]['text'].strip()
    cur_prediction = ""
    for i in reversed(tmp):
      if i.isdigit():
        cur_prediction = i + cur_prediction
      elif cur_prediction == "" or i == ",":
        continue
      else:
        break

    print("\nanswer: " + cur_anwer + ", prediction: " + result['generations'][0]['text'] + "\nextracted: " + cur_prediction + "\n")

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_5shot_CoT_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:01<01:33,  1.90s/it]


answer: 18, prediction: Janet makes 2*16 = $<<2*16=32>>32 dollars every day.
The answer is 32
extracted: 32

Acc: 0.0


  4%|▍         | 2/50 [00:05<02:20,  2.93s/it]


answer: 3, prediction: It takes 3 bolts of blue fiber and half that much white fiber.
So it takes 3 + half = <<3+half=3.5>>3.5 bolts of blue fiber and half that much white
extracted: 5

Acc: 0.0


  6%|▌         | 3/50 [00:08<02:21,  3.02s/it]

...


answer: 800, prediction: John buys 2 x 2 = <<2*2=4>>4 red ties.
Each red tie costs $50/2 = <<50/2=25>>25.
So he spent $2
extracted: 2

Acc: 0.08333333333333333


 98%|█████████▊| 49/50 [02:59<00:03,  3.86s/it]


answer: 8, prediction: The wire is 4 feet long.
The 4 feet can be divided into 2 pieces each of 2 feet
Then each 2 feet can be divided into 6 pieces each of 1 foot
extracted: 1

Acc: 0.08163265306122448


100%|██████████| 50/50 [03:03<00:00,  3.66s/it]


answer: 30, prediction: In the building, there are 15 x 8 = <<15*8=120>>120 units altogether
3/4 = <<3/4=75>>75 of them are occupied
The remaining
extracted: 75

Acc: 0.08


### Step 2-3. Implementing CoT to ten-shot Prompting

In [63]:
with open("prompt_10shot_CoT.txt", "r") as f:
    prompt_10shot_CoT = f.read()

from tqdm import tqdm

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_10shot_CoT.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break

    tmp = result['generations'][0]['text'].strip()
    cur_prediction = ""
    for i in reversed(tmp):
      if i.isdigit():
        cur_prediction = i + cur_prediction
      elif cur_prediction == "" or i == ",":
        continue
      else:
        break

    print("\nanswer: " + cur_anwer + ", prediction: " + result['generations'][0]['text'] + "\nextracted: " + cur_prediction + "\n")

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_10shot_CoT_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:03<02:53,  3.55s/it]


answer: 18, prediction: Janet sells 16-3-4 = <<16-3-4=5>>5 eggs every day for $2 each.
This means, she is able to sell 5 x $2 = <<5*2
extracted: 2

Acc: 0.0


  4%|▍         | 2/50 [00:07<02:52,  3.60s/it]


answer: 3, prediction: Fiber length = <<1/2>> = <<1/2=0.5>>0.5, blue, white
So total blue + total white = 2*0.5 + 0.5*0.5 =
extracted: 5

Acc: 0.0


  6%|▌         | 3/50 [00:10<02:38,  3.38s/it]

...


answer: 800, prediction: John spent $200 on blue ties
So he bought 200/40 = <<200/40=5>>5 blue ties
extracted: 5

Acc: 0.10416666666666667


 98%|█████████▊| 49/50 [02:44<00:03,  3.29s/it]


answer: 8, prediction: Tracy used 4*6 = <<4*6=24>>24 pieces of wire.
She cut each piece into 6 pieces. So she obtained 24*6 = <<24*6=144
extracted: 144

Acc: 0.10204081632653061


100%|██████████| 50/50 [02:48<00:00,  3.36s/it]


answer: 30, prediction: The building is 15/4 = <<15/4=3.75>>3.75 floors high.
So, there are 3.75-3.75 = <<3.75-
extracted: 75

Acc: 0.1


## **Experience 3**. Comparing CoT with different Model Scale

### Step 3-1. CoT with OPT-350M

In [88]:
with open("prompt_10shot_CoT.txt", "r") as f:
    prompt_10shot_CoT = f.read()

from tqdm import tqdm

llama = LLM("facebook/opt-350m", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_10shot_CoT.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break

    tmp = result['generations'][0]['text'].strip()
    cur_prediction = ""
    for i in reversed(tmp):
      if i.isdigit():
        cur_prediction = i + cur_prediction
      elif cur_prediction == "" or i == ",":
        continue
      else:
        break

    print("\nanswer: " + cur_anwer + ", prediction: " + result['generations'][0]['text'] + "\nextracted: " + cur_prediction + "\n")

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_10shot_CoT_direct_opt350m.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:01<00:58,  1.19s/it]


answer: 18, prediction: She buys 4.5 eggs per day
She eats 3 for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in
extracted: 2

Acc: 0.0


  4%|▍         | 2/50 [00:02<00:55,  1.15s/it]


answer: 3, prediction: Each bolt costs 1/2 the color of the robe.
Each bolt costs 1/2 the color of the robe
Each bolt costs 1/2 the color of the robe
Each bolt costs 1/2 the color of the robe
Each
extracted: 2

Acc: 0.0


  6%|▌         | 3/50 [00:03<00:53,  1.14s/it]

...


answer: 800, prediction: John bought 8 red ties, 1 red tie, and 3 blue ties.
The red ties cost $200 each.
The blue ties cost $40 each.
The 3 blue ties cost $100 each.
The 2 red ties cost $
extracted: 2

Acc: 0.020833333333333332


 98%|█████████▊| 49/50 [00:52<00:01,  1.00s/it]


answer: 8, prediction: Tracy bought 24 pieces of wire in one day.
The answer is 24
extracted: 24

Acc: 0.02040816326530612


100%|██████████| 50/50 [00:53<00:00,  1.08s/it]


answer: 30, prediction: Richard's total number of units is 8/10 = 50.
So, Richard has 10 units in his apartment, and 10/10 = 50.
There are 10 units in each floor, so he has 10/50 = 50.

extracted: 50

Acc: 0.02


### Step 3-2. CoT with OPT-1.3B

In [89]:
with open("prompt_10shot_CoT.txt", "r") as f:
    prompt_10shot_CoT = f.read()

from tqdm import tqdm

llama = LLM("facebook/opt-1.3b", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_10shot_CoT.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break

    tmp = result['generations'][0]['text'].strip()
    cur_prediction = ""
    for i in reversed(tmp):
      if i.isdigit():
        cur_prediction = i + cur_prediction
      elif cur_prediction == "" or i == ",":
        continue
      else:
        break

    print("\nanswer: " + cur_anwer + ", prediction: " + result['generations'][0]['text'] + "\nextracted: " + cur_prediction + "\n")

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_10shot_CoT_direct_opt1_3b.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:01<01:08,  1.39s/it]


answer: 18, prediction: 
Janet sells 16 eggs for $2 each and sells 4 muffins for $2 each. The result is 16 eggs for $2 x 4 = $<<16x4=60>>60 eggs.
She sells 4 muffins every day
extracted: 4

Acc: 0.0


  4%|▍         | 2/50 [00:02<01:03,  1.32s/it]


answer: 3, prediction: 
Twelve bolts of blue fiber and half that much white fiber = <<12*12=72>>72.
Twelve bolts of blue fiber and half that much white fiber = <<12*12=72>>72.
Twelve bolts
extracted: 72

Acc: 0.0


  6%|▌         | 3/50 [00:04<01:10,  1.51s/it]

...


answer: 800, prediction: John spent $40/50 = <<40/50=$40/50=60>>60 ties
So he spent $40/50 = <<40/50=$40/50=120>>120 ties on red ties.
John
extracted: 120

Acc: 0.020833333333333332


 98%|█████████▊| 49/50 [01:14<00:01,  1.66s/it]


answer: 8, prediction: She obtained 6 pieces
Each piece has 24 inches of wire.
Thus, she obtained 24/(24+6) = 12.
Each piece represents 24 hours, so each hour of wire represents 24 * 12 = 48 hours
Each wire represents
extracted: 48

Acc: 0.02040816326530612


100%|██████████| 50/50 [01:15<00:00,  1.52s/it]


answer: 30, prediction: There are 255 unoccupied units in the building, and 66 are vacant.
The total number of units = 255*3/4 = <<255*3/4=42>>42
The total number of units in the building is 255/
extracted: 255

Acc: 0.02


### Step 3-3. CoT with OPT-2.7B

In [90]:
with open("prompt_10shot_CoT.txt", "r") as f:
    prompt_10shot_CoT = f.read()

from tqdm import tqdm

llama = LLM("facebook/opt-2.7b", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_10shot_CoT.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break

    tmp = result['generations'][0]['text'].strip()
    cur_prediction = ""
    for i in reversed(tmp):
      if i.isdigit():
        cur_prediction = i + cur_prediction
      elif cur_prediction == "" or i == ",":
        continue
      else:
        break

    print("\nanswer: " + cur_anwer + ", prediction: " + result['generations'][0]['text'] + "\nextracted: " + cur_prediction + "\n")

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_10shot_CoT_direct_opt2_7b.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:01<01:11,  1.47s/it]


answer: 18, prediction: John ate four eggs for breakfast every day. He bought two dozen each at the farmers' market.
He bought 24 ducks for $2 each at the farmers' market.
So he bought 24 / 2 = <<24/2=12>>
extracted: 12

Acc: 0.0


  4%|▍         | 2/50 [00:03<01:30,  1.88s/it]


answer: 3, prediction: It takes $16 bolts in blue fiber and $32 bolts in white fiber to make a robe.
Let B be the number of bolts in blue fiber and C be the number of bolts in white fiber.
Since B = C, the number
extracted: 32

Acc: 0.0


  6%|▌         | 3/50 [00:05<01:34,  2.00s/it]

...


answer: 800, prediction: John spent $200 on ties that cost $40 each.
He spent $40 * 2 = <<40*2=80>>80 + $200 = $<<80*2=160>>160
He spent 80 + 80 = <<80
extracted: 80

Acc: 0.041666666666666664


 98%|█████████▊| 49/50 [01:20<00:01,  1.44s/it]


answer: 8, prediction: She obtained 4 pieces from each of the 4 pieces of wire.
In this case, she obtained 4 pieces from each piece of wire = 4*4 = 8 pieces.
She obtained 8 pieces from each piece of wire = 8*8 =
extracted: 8

Acc: 0.061224489795918366


100%|██████████| 50/50 [01:22<00:00,  1.65s/it]


answer: 30, prediction: The total number of units in the building is 15 + 3/4 = <<15+3/4=18>>18.
The total number of units is 15 + 3/4 = <<15+3/4=18>>18

extracted: 18

Acc: 0.06
